In [31]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
print("Device:", device)

Device: cpu


In [21]:
with open('/content/drive/MyDrive/train.pkl', 'rb') as f:
    data_set = pickle.load(f)

In [24]:
data_set[0][1]

0

In [25]:
data = []
targets = []
for i in data_set:
  data.append(i[0])
  targets.append(i[1])

In [27]:
print(data[1000])
print(targets)

[ 67.  12.  12. 159.  12.  28.  28.  12.  13.  12.  28.  12.  28.  28.
  28.  28.  12.  29. 125.  28.  28.  12. 159.  12.  15.  13.  79.  28.
  78.  28.  28.  12.  28.  28.  47.   0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
from collections import Counter

labels = [label for _, label in data]

class_counts = Counter(labels)
classes = []
print("Liczba klas:", len(class_counts))
for class_label, count in class_counts.items():
    classes.append(count)
classes = np.array(classes)
print(classes)

Liczba klas: 5
[1630  478  154  441  236]


In [18]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [58]:
train_indices = np.random.rand(len(data_set))>0.3

In [59]:
train_indices

array([False,  True, False, ...,  True,  True, False])

In [73]:
data_train = []
targets_train = []
data_test = []
targets_test = []
for i in range(len(data_set)):
  if train_indices[i] == True:
    data_train.append(data_set[i][0])
    targets_train.append(data_set[i][1])
  else:
    data_test.append(data_set[i][0])
    targets_test.append(data_set[i][1])

4


In [44]:
# dataset_size = len(data_set)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size


# train_set = VariableLenDataset(data[:train_indices], targets[:train_indices])
# test_set = VariableLenDataset(data[train_indices:], targets[train_indices:])

In [74]:
train_set = VariableLenDataset(data_train, targets_train)
test_set = VariableLenDataset(data_test, targets_test)

In [75]:
print(train_set[2])

(array([144., 144., 144., 144., 144.,  64.,  64.,  64.,   0., 112., 112.,
       112., 112., 176., 176., 176.,  82.,   0.,   0.,   0.,  82., 176.,
        66., 144.,  34., 128., 128., 128., 128., 131.,  65.,  18.,  50.,
       144.,  68.,   0.,  68.,  34.,  68.,  52.,   0.,   4.,   4.,   0.,
       106.,   0.,  68., 131.,  36.,  81.,  84.,  66.,  88.,  67.,  12.,
        35., 148.,   3.,   0.,   2., 100.,   6.,   6.,   8.,  12.,  69.,
        38.,  34., 152., 146.,  40.,  24., 159., 110., 122., 112.,  26.,
       112., 189.,  90.,  61.,   4.,  76.,  92., 124., 112., 189., 183.,
       111., 137.,  12.,   1.,  42.,   3.,  77., 138., 124., 144.,  25.,
        30.,  44.,  33.,  47.,   3.,  12., 177., 124.,  34.,  37.,  64.,
         6., 148.,  12., 152., 111.,  73., 157.,  37., 124., 150.,  88.,
        72.,  62., 106., 106.,   0.,  74.,  42.,  78.,  43., 156., 172.,
        47., 106., 141.,  71.,  12.,  69.,  77.,  77.,  92.,   8.,  12.,
        47.,  78.,  64.,  69.,  67.,  88.,  33.,  

In [76]:
pad = -100

# def pad_collate(batch, pad_value=pad):
#     xx, yy = zip(*batch)
#     xx = [torch.tensor(x) for x in xx]
#     x_lens = [len(x) for x in xx]
#     yy_pad = torch.tensor(yy)
#     y_lens = [1] * len(yy)
#     # print(xx)
#     # print(yy)

#     xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
#     # yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

#     return xx_pad, yy_pad, x_lens, y_lens

def pad_collate(batch, pad_value=pad):
    try:
        xx, yy = zip(*batch)
    except ValueError as e:
        print(f"Error while unpacking batch: {e}")
        print(f"Batch content: {batch}")
        raise

    xx = [torch.tensor(x) for x in xx]
    x_lens = [len(x) for x in xx]
    yy_pad = torch.tensor(yy)
    y_lens = [1] * len(yy)

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [97]:
batch_size=64

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [30]:
# dataset_size = len(data)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size

# train_set, test_set = random_split(data, [train_size, test_size])

# batch_size=4
# trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [39]:
data_set[0]


(array([ -1.,  -1.,  -1., ...,  78.,  40., 144.]), 0)

In [12]:
print(len(train_set[0][0]))
print(len(train_set[1][0]))

108
810


In [82]:
next(iter(train_loader))

(tensor([[ 146.,   25.,  180.,  ..., -100., -100., -100.],
         [  -1.,   82.,  140.,  ..., -100., -100., -100.],
         [  -1.,   -1.,   -1.,  ..., -100., -100., -100.],
         ...,
         [  -1.,   -1.,   -1.,  ..., -100., -100., -100.],
         [  64.,   64.,  144.,  ..., -100., -100., -100.],
         [  34.,  156.,   30.,  ..., -100., -100., -100.]], dtype=torch.float64),
 tensor([0, 1, 0, 0, 0, 0, 4, 2, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 2, 0, 0, 4, 3, 0,
         1, 0, 0, 0, 3, 0, 1, 0, 0, 3, 0, 0, 1, 4, 0, 2, 0, 0, 0, 0, 3, 0, 0, 1,
         0, 3]),
 [192,
  189,
  788,
  140,
  138,
  76,
  316,
  748,
  172,
  2128,
  68,
  68,
  86,
  2860,
  457,
  69,
  347,
  36,
  206,
  192,
  189,
  206,
  204,
  525,
  725,
  186,
  52,
  356,
  348,
  126,
  52,
  713,
  452,
  1232,
  480,
  96,
  363,
  148,
  135,
  132,
  222,
  68,
  68,
  432,
  171,
  64,
  51,
  72,
  552,
  224],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [100]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size*90*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        # x = torch.transpose(x,0,1)
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        # all_outputs = torch.transpose(all_outputs,0,1)
        out = torch.flatten(all_outputs,1)
        x = self.fc(out)
        return x, hidden

model = LSTMRegressor(1,5,2,5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=450, out_features=5, bias=True)
)

In [101]:
with torch.no_grad():
    for x, targets, x_len, target_len in test_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[0])
        hidden, state = hidden.to(device), state.to(device)

#         x = torch.transpose(x, 0, 1)
#         preds, _ = model(x, (hidden, state))
#         preds = torch.transpose(preds, 0, 1)

        x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        # print(x_packed)
        preds_packed, _ = model(x_packed, (hidden, state))
        preds, pred_len = pad_packed_sequence(preds_packed, batch_first=True, padding_value=pad)



        preds = preds.squeeze(2)
        mask_tgt = targets != pad
        print(targets)
        print(preds)
        print("Targets shape:", targets.shape)
        print("Preds shape:", preds.shape)
        print("Mask shape:", mask_tgt.shape)
        print(torch.abs(preds[mask_tgt] - targets[mask_tgt]).mean())
        print()

TypeError: flatten(): argument 'input' (position 1) must be Tensor, not PackedSequence

In [ ]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds,_ =  model(test_data.to(device).unsqueeze(2), (hidden,state))
print(f"Accuracy: {(torch.argmax(preds,1).cpu()==test_targets).sum().item()/len(test_targets):.3}")